In [1]:
import tensorflow as tf
import tensorflow_addons as tfa  # for potential advanced augmentations
# import tensorflow_probability as tfp  # if using MixUp/CutMix


C:\Users\ASUS\anaconda3\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:

# Constants\IMAGE_SIZE = 256
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
NUM_CLASSES = 3
IMAGE_SIZE= 256
DATA_DIR = "PlantVillage"


In [5]:

#----------------------------------------
# 1. DATA LOADING
#----------------------------------------
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels='inferred',
    label_mode='int',
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)


Found 5702 files belonging to 3 classes.
Using 4562 files for training.


In [6]:

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels='inferred',
    label_mode='int',
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)


Found 5702 files belonging to 3 classes.
Using 1140 files for validation.


In [7]:

#----------------------------------------
# 2. PREPROCESSING & AUGMENTATION
#----------------------------------------
# 2a) Basic Resize + Rescale
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    tf.keras.layers.Rescaling(1./255)
])


In [8]:

# 2b) On-the-fly Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(factor=0.2),
    tf.keras.layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    tf.keras.layers.RandomContrast(factor=0.2),
])


In [9]:

# 2c) Dataset formatting function

def format_train(image, label):
    image = resize_and_rescale(image)
    image = data_augmentation(image)
    return image, label


In [10]:

# Apply mapping
train_ds = (train_ds
            .map(format_train, num_parallel_calls=AUTOTUNE)
            .cache()
            .prefetch(AUTOTUNE))

val_ds = (val_ds
          .map(lambda x, y: (resize_and_rescale(x), y), num_parallel_calls=AUTOTUNE)
          .cache()
          .prefetch(AUTOTUNE))


In [11]:

#----------------------------------------
# 3. MIXUP / CUTMIX EXAMPLE (optional)
#----------------------------------------
"""
# Uncomment if using MixUp or CutMix; requires TensorFlow Probability for Beta sampling
import tensorflow_probability as tfp

def sample_beta_distribution(size, concentration_0=0.2, concentration_1=0.2):
    dist = tfp.distributions.Beta(concentration_0, concentration_1)
    return dist.sample(sample_shape=[size])


def mixup(images, labels, alpha=0.2):
    batch_size = tf.shape(images)[0]
    beta = sample_beta_distribution(batch_size, alpha, alpha)
    x_weight = tf.reshape(beta, (batch_size, 1, 1, 1))
    lab_weight = tf.reshape(beta, (batch_size, 1))
    index = tf.random.shuffle(tf.range(batch_size))
    mixed_images = images * x_weight + tf.gather(images, index) * (1 - x_weight)
    mixed_labels = labels * lab_weight + tf.gather(labels, index) * (1 - lab_weight)
    return mixed_images, mixed_labels

# To integrate:
# train_ds = train_ds.map(lambda x, y: mixup(x, tf.one_hot(y, NUM_CLASSES)), num_parallel_calls=AUTOTUNE)
"""


'\n# Uncomment if using MixUp or CutMix; requires TensorFlow Probability for Beta sampling\nimport tensorflow_probability as tfp\n\ndef sample_beta_distribution(size, concentration_0=0.2, concentration_1=0.2):\n    dist = tfp.distributions.Beta(concentration_0, concentration_1)\n    return dist.sample(sample_shape=[size])\n\n\ndef mixup(images, labels, alpha=0.2):\n    batch_size = tf.shape(images)[0]\n    beta = sample_beta_distribution(batch_size, alpha, alpha)\n    x_weight = tf.reshape(beta, (batch_size, 1, 1, 1))\n    lab_weight = tf.reshape(beta, (batch_size, 1))\n    index = tf.random.shuffle(tf.range(batch_size))\n    mixed_images = images * x_weight + tf.gather(images, index) * (1 - x_weight)\n    mixed_labels = labels * lab_weight + tf.gather(labels, index) * (1 - lab_weight)\n    return mixed_images, mixed_labels\n\n# To integrate:\n# train_ds = train_ds.map(lambda x, y: mixup(x, tf.one_hot(y, NUM_CLASSES)), num_parallel_calls=AUTOTUNE)\n'

In [12]:

#----------------------------------------
# 4. MODEL DEFINITION
#----------------------------------------
def get_model(model_name="EfficientNetB0", input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), num_classes=NUM_CLASSES):
    if model_name == "ResNet50":
        base = tf.keras.applications.ResNet50(
            include_top=False, input_shape=input_shape, weights='imagenet', pooling='avg')
    elif model_name == "DenseNet121":
        base = tf.keras.applications.DenseNet121(
            include_top=False, input_shape=input_shape, weights='imagenet', pooling='avg')
    elif model_name == "MobileNetV2":
        base = tf.keras.applications.MobileNetV2(
            include_top=False, input_shape=input_shape, weights='imagenet', pooling='avg')
    else:  # Default to EfficientNetB0
        base = tf.keras.applications.EfficientNetB0(
            include_top=False, input_shape=input_shape, weights='imagenet', pooling='avg')

    # Freeze base
    base.trainable = False

    inputs = tf.keras.Input(shape=input_shape)
    x = base(inputs, training=False)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, outputs)


In [13]:
# Instantiate and compile
model = get_model(model_name="EfficientNetB0")
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


16705208/16705208 [==============================] - 5s 0us/step


In [ ]:


#----------------------------------------
# 5. TRAINING
#----------------------------------------
EPOCHS = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/20
143/143 [==============================] - 231s 2s/step - loss: 1.1271 - accuracy: 0.3273 - val_loss: 1.0999 - val_accuracy: 0.3237
Epoch 2/20
143/143 [==============================] - 197s 1s/step - loss: 1.1226 - accuracy: 0.3332 - val_loss: 1.0998 - val_accuracy: 0.3237
Epoch 3/20
143/143 [==============================] - 197s 1s/step - loss: 1.1216 - accuracy: 0.3317 - val_loss: 1.0993 - val_accuracy: 0.3237
Epoch 4/20
143/143 [==============================] - 195s 1s/step - loss: 1.1261 - accuracy: 0.3345 - val_loss: 1.0997 - val_accuracy: 0.3237
Epoch 5/20
143/143 [==============================] - 202s 1s/step - loss: 1.1172 - accuracy: 0.3448 - val_loss: 1.0993 - val_accuracy: 0.3237
Epoch 6/20
143/143 [==============================] - 200s 1s/step - loss: 1.1186 - accuracy: 0.3336 - val_loss: 1.0992 - val_accuracy: 0.3237
Epoch 7/20
143/143 [==============================] - 194s 1s/step - loss: 1.1189 - accuracy: 0.3415 - val_loss: 1.0995 - val_accuracy: 0.3237

In [15]:


# Optional: Unfreeze some layers for fine-tuning
# base = model.layers[1]
# base.trainable = True
# for layer in base.layers[:-20]: layer.trainable = False
# model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
# history_ft = model.fit(train_ds, validation_data=val_ds, epochs=10)

#----------------------------------------
# 6. SAVE MODEL
#----------------------------------------
model.save("models/plant_disease_model_EfficientNetB0.h5")


C:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# 1. Constants – adjust to your values
IMAGE_SIZE = 224  # must match IMAGE_SIZE you used during training
MODEL_PATH = "models/plant_disease_model_EfficientNetB0.h5"
CLASS_NAMES = [
    "Apple___Apple_scab",
    "Apple___Black_rot",
    "Apple___Cedar_apple_rust",
    "Apple___healthy",
    # … all your other class names in the same order you used during training
]

from tensorflow.keras.layers import DepthwiseConv2D as _BaseDepthwiseConv2D
from tensorflow.keras.models import load_model

# 1) Subclass the built‑in to accept & discard groups
class DepthwiseConv2D(_BaseDepthwiseConv2D):
    def __init__(self, *args, groups=1, **kwargs):
        # ignore groups, pass everything else through
        super().__init__(*args, **kwargs)

# 2) Now load with custom_objects
MODEL_PATH = "models/plant_disease_model_EfficientNetB0.h5"
model = load_model(
    MODEL_PATH,
    custom_objects={"DepthwiseConv2D": DepthwiseConv2D}
)

print("✅ Model loaded successfully with custom DepthwiseConv2D")

# 3. Pre‐processing helper
def preprocess_img(img_path: str) -> np.ndarray:
    """
    Loads an image file, resizes to (IMAGE_SIZE, IMAGE_SIZE), scales pixels to [0,1],
    and adds batch dimension.
    """
    img = image.load_img(img_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = image.img_to_array(img)               # shape: (H, W, 3)
    img_array /= 255.0                                # scale to [0,1]
    return np.expand_dims(img_array, axis=0)    # 4. Single‐image inference
test_img = "PlantVillage/Potato___Early_blight/0a8a68ee-f587-4dea-beec-79d02e7d3fa4___RS_Early.B 8461.JPG"  # replace with your image path  # replace with your image path
x = preprocess_img(test_img)
preds = model.predict(x)                    # shape: (1, num_classes)
predicted_index = np.argmax(preds[0])
predicted_label = CLASS_NAMES[predicted_index]
confidence = preds[0][predicted_index]

print(f"Image: {test_img}")
print(f"Predicted class: {predicted_label}  (confidence: {confidence:.3f})")

# 5. Batch inference on a folder
test_folder = "PlantVillage\Potato___Early_blight"
for fname in os.listdir(test_folder):
    if fname.lower().endswith((".jpg", ".png", ".jpeg")):
        img_path = os.path.join(test_folder, fname)
        x = preprocess_img(img_path)
        preds = model.predict(x)
        idx = np.argmax(preds[0])
        label = CLASS_NAMES[idx]
        conf = preds[0][idx]
        print(f"{fname:20s} → {label:30s} ({conf:.3f})")


In [3]:
import os
import tensorflow as tf

# ——— 0) (Optional) Restrict visible GPUs — e.g. use only GPU 0
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ——— 1) List and configure GPUs
gpus = tf.config.list_physical_devices('GPU')
if not gpus:
    raise RuntimeError("No GPU found. Check your CUDA/cuDNN installation and that you're using a GPU-enabled TF build.")
# Enable “memory growth” so TF only allocates what it needs
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

print("GPUs detected:", gpus)

# ——— 2) (Re-)load your model — with the monkey‑patch custom DepthwiseConv2D if needed
from tensorflow.keras.layers import DepthwiseConv2D as _BaseDepthwiseConv2D
from tensorflow.keras.models import load_model

class DepthwiseConv2D(_BaseDepthwiseConv2D):
    def __init__(self, *args, groups=1, **kwargs):
        super().__init__(*args, **kwargs)

model = load_model(
    "models/plant_disease_model_EfficientNetB0.h5",
    custom_objects={"DepthwiseConv2D": DepthwiseConv2D}
)
print("Model loaded. Default device:", tf.test.gpu_device_name())

# ——— 3) Wrap inference in the GPU device context
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess_img(path, img_size=224):
    img = image.load_img(path, target_size=(img_size, img_size))
    arr = image.img_to_array(img) / 255.0
    return np.expand_dims(arr, 0)

test_img = "PlantVillage/Potato___Early_blight/0a8a68ee-f587-4dea-beec-79d02e7d3fa4___RS_Early.B 8461.JPG"
x = preprocess_img(test_img)

# Make sure to run on GPU:0
with tf.device('/GPU:0'):
    preds = model.predict(x)

idx = np.argmax(preds[0])
print(f"Predicted: {CLASS_NAMES[idx]}  (conf: {preds[0][idx]:.3f})")


RuntimeError: No GPU found. Check your CUDA/cuDNN installation and that you're using a GPU-enabled TF build.

In [1]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())        # True if built with CUDA
print(tf.config.list_physical_devices('GPU'))  # Lists available GPUs


False
[]


In [2]:
!pip install --upgrade pip
!pip install tensorflow  # as of TF 2.15 this includes GPU support for CUDA 11.8


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [ ]:
!pip install --upgrade pip

In [5]:
!conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0

^C
